# 1.1 Download Artigos SciELO

Neste notebook é possível fazer download dos artigos da RBEF que podem ser encontrados no site de [SciELO](https://www.scielo.br/j/rbef/). 

## Imports

In [1]:
import os
from urllib.parse import urljoin

from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm

from functions import Request

In [2]:
request = Request()  # objeto de requisição

## Funções

In [3]:
scielo_url = lambda x: urljoin('https://www.scielo.br', x)  # Função geradora de urls

In [4]:
def volumes_por_ano() -> dict[int:list]:
    """
        Função que pega o link de cada volume da revista por ano.

    Returns:
        dict: Dicionário onde as chaves é o ano e os valores os links dos volumes do respectivo ano.
    """
    req = request.get(scielo_url('j/rbef/grid'))
    soup = BeautifulSoup(req.text, 'lxml', parse_only=SoupStrainer('tbody'))

    links = {int(linha.td.text): [scielo_url(a['href']) for a in linha.find_all('a')] for linha in soup.find_all('tr')}
    
    return links

In [5]:
def artigos_por_volume(url: str, formato: str = 'html', idioma: str = 'pt') -> list[str]:
    """
        Função que coleta o link de todos os artigos presentes no volume.

    Args:
        url (str): url do volume
        formato (str, optional): Formato dos artigos. Defaults to 'html'.
        idioma (str, optional): Idioma dos artigos. Defaults to 'pt'.

    Returns:
        list: Lista com os links dos artigos deste volume.
    """
    languages = {'pt': 'Português', 'en': 'Inglês', 'es': 'Espanhol'}
    formatos = {'html': '/?lang=', 'pdf': '/?format=pdf&'}

    lang = languages.get(idioma.lower())
    form = formatos.get(formato.lower())

    req = request.get(url)

    soup = BeautifulSoup(req.text, 'lxml', parse_only=SoupStrainer('a', attrs={'title': lang}))

    return [scielo_url(a['href']) for a in soup.find_all('a') if 'abstract' not in a['href'] and form in a['href']]

In [6]:
def html_do_artigo(url: str) -> BeautifulSoup:
    """
        Função que recebe uma url de um artigo e retorna uma string do seu html.

    Args:
        url (str): Url do artigo

    Returns:
        str: Retorna uma string do html da página do artigo.
    """
    req = request.get(url)
    
    strainer = SoupStrainer('div', attrs={'class': 'articleTxt'})
    
    return BeautifulSoup(req.text, 'lxml', parse_only=strainer).prettify()

## Código

Nesta parte do notebook consiste no código de download dos artigos que estão no site da SciELO e que são da língua portugesa.

In [7]:
volumes = volumes_por_ano() 
del volumes[2023]  # remove 2023 dos volumes.

In [8]:
try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos')) # cria a pasta dos artigos brutos
except FileExistsError:
    print('A pasta `artigos_brutos` já existe!')

try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos', 'SciELO')) # cria a pasta da SciELO dentro dos artigos brutos
except FileExistsError:
    print('A pasta `artigos_brutos\SciELO` já existe!')

A pasta `artigos_brutos` já existe!
A pasta `artigos_brutos\SciELO` já existe!


In [9]:
path = os.path.join(os.getcwd(), 'artigos_brutos', 'SciELO')

In [10]:
n = 0
for ano, urls in tqdm(volumes.items(), desc='Anos'):
    for volume_url in urls:
        todos = artigos_por_volume(volume_url)
        for artigo_url in todos:
            nome_arquivo = str(n).rjust(4, "0") + '.html'
            path_save = os.path.join(path, nome_arquivo)
            
            with open(path_save, 'w', encoding='utf8') as file:
                file.write(html_do_artigo(artigo_url))
            
            n += 1

Anos: 100%|██████████| 22/22 [18:02<00:00, 49.22s/it]


Depois de fazer o download dos arquivos e salvá-los em uma pasta, podemos criar um Dataset para facilitar a navegação pelo conteúdo.

Após a criação do dataset esses arquivos brutos são apagados.